In [48]:
import tensorflow as tf
import numpy as np

# preprocessed data
#from datasets.twitter import data
import data
import data_utils
import os

print(tf.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

1.6.0


In [82]:
metadata, idx_q, idx_a = data.load_data(PATH='')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [85]:

# parameters 
xseq_len = trainX.shape[-1]
print(xseq_len)
print(trainY[1])
yseq_len = trainY.shape[-1]
print(yseq_len)
batch_size = 24
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024
print (xvocab_size)
import seq2seq_wrapper

40
[ 10   2   8  88   4  48   3 410  15 114   6  13   9  71   6  36  67   9
  27   3  41   2  17  26   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]
40
2153


In [86]:
print(trainX.shape)
print(validX.shape)
print(testX.shape)


(329, 40)
(70, 40)
(70, 40)


In [87]:
setattr(tf.contrib.rnn.GRUCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.BasicLSTMCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.MultiRNNCell, '__deepcopy__', lambda self, _: self)
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )


<log> Building Graph </log>

In [88]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 24)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [94]:
sess = model.restore_last_session()
sess = model.train(train_batch_gen, val_batch_gen)

INFO:tensorflow:Restoring parameters from ckpt/twitter/seq2seq_model.ckpt-40

<log> Training started </log>

Model saved to disk at iteration #10
val   loss : 5.197876

Model saved to disk at iteration #20
val   loss : 4.381861

Model saved to disk at iteration #30
val   loss : 4.078625

Model saved to disk at iteration #40
val   loss : 3.977541

Model saved to disk at iteration #50
val   loss : 4.009060

Model saved to disk at iteration #60
val   loss : 3.992224

Model saved to disk at iteration #70
val   loss : 3.957519

Model saved to disk at iteration #80
val   loss : 3.934952

Model saved to disk at iteration #90
val   loss : 3.980436

Model saved to disk at iteration #100
val   loss : 3.943811

Model saved to disk at iteration #110
val   loss : 3.958645

Model saved to disk at iteration #120
val   loss : 3.983100

Model saved to disk at iteration #130
val   loss : 3.847417

Model saved to disk at iteration #140
val   loss : 3.762195

Model saved to disk at iteration #150
val   lo

In [90]:
sess = model.restore_last_session()
print(validX)

INFO:tensorflow:Restoring parameters from ckpt/twitter/seq2seq_model.ckpt-40
[[  32  110    7 ...    0    0    0]
 [ 298  187 1076 ...    0    0    0]
 [  32  354  720 ...    0    0    0]
 ...
 [  32   56   21 ...    0    0    0]
 [ 152   23    0 ...    0    0    0]
 [  14   60   84 ...    0    0    0]]


In [91]:
res_idx = model.predict(sess, validX)
print(res_idx)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 2 2 ... 0 0 0]
 [2 2 2 ... 0 0 0]
 [2 2 2 ... 0 0 0]]


In [92]:
import pickle
def idx_2_text(result):
    # read data control dictionaries
    with open('metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
        idx2wd = metadata['idx2w']
#        for i in result:
            #print(idx2wd[i])
    
    return idx2wd

In [93]:
i=0
j=0
rep_all = [[]]
idx2word = idx_2_text(k)
for reply in res_idx:
#    print(reply)
    rep_all.append([])
    for k in reply:
        #print(k, idx2word[k])
        rep_all[j].append(idx2word[k])
    #print(rep_all)
    j = j+1
#    i= i+1
    
print(rep_all)

[['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['you', 'you', 'you', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['you', 'you', 'you', 'you', 'you', 'you', 'you', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 